# Homework1: Part 2

## Junkyard Quest

Complete the quest and escape from this abandoned junkyard! Player must find a way to escape in time in order to reconnect with man kind! 

### Import Text Adventures

In [1]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

## Locations

In [2]:

phone_booth = things.Location(
    "Phone booth",
    "You are standing in a rusted Phonebooth."
)
gravel_path = things.Location(
    "Gravel Path",
    "You are standing on a dusty gravel path, there is a Junkyard here",
)
junkyard = things.Location(
    "Junkyard",
    "You are in a junkyard."
)
dirt_path = things.Location(
    "dirt Path",
    "You are walking along a dirt path. There is a garage here.",
)
garage = things.Location(
    "garage",
    "You are in the garage."
)
concrete_path = things.Location(
    "concrete path",
    "You are standing on the concrete path leading to the Office.",
)
office = things.Location(
    "Office",
    "You are in the Office."
)
death = things.Location(
    "The Afterlife",
    "You are dead. GAME OVER."
)
death.set_property("game_over", True)



# Map of Locations
gravel_path.add_connection("south",junkyard)
gravel_path.add_connection("north", phone_booth)
junkyard.add_connection("north", gravel_path)
junkyard.add_connection("east",dirt_path)
garage.add_connection("west", dirt_path)
dirt_path.add_connection("west",junkyard)
junkyard.add_connection("west", concrete_path)
concrete_path.add_connection("west", office)
concrete_path.add_connection("east", junkyard)





###  Preliminary Game Map

Here is a visualization of the locations in action castle represented as a directed graph.

![Visualization of game map starting from the tower](../map-from-tower.svg)

In [ ]:
tempgame = games.Game(phone_booth, things.Character(name="Temp Player", description="", persona=""))

from text_adventure_games.viz import Visualizer
viz = Visualizer(tempgame)
graph = viz.visualize()
graph

## Items

### Gettable Items

In [4]:
# screwdriver in the phone booth
screw_driver= things.Item(
    "screw driver",
    "a screwdriver",
    "A SIMPLE SCREW DRIVER.",
)
phone_booth.add_item(screw_driver)


# keys to the cars in the junkyard 
car_key = things.Item(
    "car key",
    "a single car key",
    "IT LOOKS THIS BELONGS TO A CAR.",
)
office.add_item(car_key)

#keys to unlock the office door
door_key = things.Item(
    "door key", 
    "A single door key", 
    "IT LOOKS LIKE THIS KEY COULD FIT IN A DOOR KNOB"
)
garage.add_item(door_key)


# fuel can for the cars in the junkyard 
fuel_can = things.Item(
    "fuel can",
    "a fuel can filled with gasoline",
    "THE FUEL CAN CONTAINS GASOLINE FOR CARS.",
)
garage.add_item(fuel_can)

fuel_can.add_command_hint("fill into car")
car_key.add_command_hint("start car")


### Scenery Items

_We called these "non-gettable" in the tutorial_

In [5]:


# A rusted car that needs fuel and keys
rusted_car = things.Item(
    "rusted car",
    "a rusted old car",
    "THE RUSTED CAR LOOKS LIKE IT COULD DRIVE.",
)
rusted_car.set_property("gettable", False)
rusted_car.set_property("has_fuel", False)
rusted_car.set_property("has_key", False)
rusted_car.set_property("running", False)
rusted_car.add_command_hint("Put in fuel")
rusted_car.add_command_hint("Put in key")
junkyard.add_item(rusted_car)


# A door that leads to the office
door = things.Item(
    "door",
    "a door",
    "THE DOOR IS SECURELY LOCKED."
)
door.set_property("gettable", False)
door.set_property("is_locked", True)
door.add_command_hint("unlock door")
concrete_path.add_item(door)


## Characters

### Player

In [6]:
# Player
player = things.Character(
    name="The player",
    description="You are a simple peasant destined for greatness.",
    persona="I am on an adventure.",
)

# Player's flash light
flashlight = things.Item("flashlight", "a flashlight", "A the flash who creates light.")
flashlight.set_property("is_lightable", True)
flashlight.set_property("is_lit", False)
flashlight.add_command_hint("light lamp")
player.add_to_inventory(flashlight)
player.set_property("driving_car", False)
player.set_property("electricuted", False)

## Actions

The player can:

- Unlock a door
- Put fuel into a car 
- Put keys into a door or a car
- Use the screwdriver to fix the car 

In [7]:
class Unlock_Door(actions.Action):
    ACTION_NAME = "unlock door"
    ACTION_DESCRIPTION = "Unlock a door with a key"
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.key = door_key
        self.door = door

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * There must be a door
        * The character must be at the same location as the door
        * The door must be locked
        * The character must have the key in their inventory
        """
        if self.door and self.character.location.here(self.door):
            if self.key and self.door.get_property("is_locked"): 
                return True
            else: 
                return False
        
        return False  

    def apply_effects(self):
        """
        Effects:
        * Unlocks the door
        """
        door.set_property("is_locked",False)
        self.parser.ok("Door is unlocked")

In [8]:
class Enter_fuel(actions.Action):
    """
    Entering fuel into the car can help the car turn on. Once the car's engine is on, the car can be driven
    """
    ACTION_NAME = "enter fuel"
    ACTION_DESCRIPTION = "Enters fuel into vehicle"
    ACTION_ALIASES = []

    def __init__(self, game, command):
        super().__init__(game)
        self.character: things.Character = self.parser.get_character(command)
        self.fuel_can: things.Item = fuel_can
        self.rusted_car: things.Item = rusted_car        

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * There must be a fuel can
        * The character must have the fuel can in their inventory
        * the rusted car must be here (junkyard)
        """
        if self.fuel_can and self.character.is_in_inventory(self.fuel_can):
            if self.character.location.here(self.rusted_car):
                return True

        return False

    def apply_effects(self):
        """
        Effects:
        * Enters fuel into the car
        """
        self.rusted_car.set_property("has_fuel", True)
        print("Fuel has been entered into the car.")
        print("Car now has sufficient Fuel to start.")


In [9]:
class Enter_key(actions.Action):
    """
    Enter the car keys into the ignition. Doing this without adding fuel will result in the car exploding.
    """
    ACTION_NAME = "enter key"
    ACTION_DESCRIPTION = "Enter key into car's ignition"
    ACTION_ALIASES = []


    def __init__(self, game, command):
        super().__init__(game)
        self.character: things.Character = self.parser.get_character(command)
        self.car_key: things.Item = car_key
        self.rusted_car: things.Item = rusted_car

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The car key must exist and be in inventory 
        * The rusted car must be present in the location
        """
        if self.car_key and self.character.is_in_inventory(self.car_key):
            if self.character.location.here(self.rusted_car):
                return True

        return False

    def apply_effects(self):
        """
        Effects:
        * The car now has keys in it's ignition. 
        """
        self.rusted_car.set_property("has_key",True)
        print("The key has been placed into the car's ignition.")
        print("The key can turn the car on.")



In [10]:
class Start_car(actions.Action):
    ACTION_NAME = "start car"
    ACTION_DESCRIPTION = "Start the vehicle in hopes to drive and escape the spooky junkyard"
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        self.character: things.Character = self.parser.get_character(command)
        self.rusted_car: things.Item = rusted_car
        self.fuel_can: things.Item = fuel_can
        self.car_key: things.Item = car_key

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The car must exist
        * The car must have either the the key or fuel
        * The player must be at the car's location 
        """
        if self.rusted_car: 
            if self.rusted_car.get_property("has_fuel") or self.rusted_car.get_property("has_key"):
                return True 

        return False

    def apply_effects(self):
        if self.rusted_car.get_property("has_fuel") and self.rusted_car.get_property("has_key"):
            self.rusted_car.set_property("running", True)
            self.character.set_property("driving_car", True)
            print("You have successfully started the car!")
        if self.rusted_car.get_property("has_key") and (not(self.rusted_car.get_property("has_fuel"))):
            print("Uh oh... starting the car without fuel caused the keys to have electrical charge...")
            self.character.set_property("electricuted", True)



## Blocks

In text adventure games, _blocks_ are essentially puzzles to unlock the next stage in the game.

There is a locked door that blocks the player from entering the office. 

In [11]:
class Door_Block(blocks.Block):
    """
    Blocks progress in this direction until the character unlocks the door.
    """

    def __init__(self, location: things.Location, door: things.Item):
        super().__init__("A locked door blocks your way", "The door ahead is locked.")
        self.location = location
        self.door = door

    def is_blocked(self) -> bool:
        # Conditions of block:
        # * The door is locked

        if self.door.get_property("is_locked"):
            return True
        if not self.door.get_property("is_locked"):
            return False
        return True

In [12]:
# adding the block in its location

Door_block = Door_Block(concrete_path, door)
concrete_path.add_block("west", Door_block)


## Play Action Castle

In [13]:
class ActionCastle(games.Game):
    def __init__(
        self, start_at: things.Location, player: things.Character, characters=None,
        custom_actions=None
    ):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)

    def is_won(self) -> bool:
        """ 
        Checks whether the game has been won. The game has been won if the player has started the car
        and driven away. 
        """
        if self.player.get_property("electricuted"):
            "Player was electricuted.... Game over."
            return True 

        for name, character in self.characters.items():
            if character.get_property("driving_car"):
                msg = "{name} is driving the car and escaping the spooky junkyard \n {name} has won the game!"
                self.parser.ok(msg.format(name=character.name.title()))
                return True
        return False

In [14]:
characters = []
custom_actions = [Unlock_Door, Enter_key, Enter_fuel, Start_car]

# The Game
game = ActionCastle(phone_booth, player, characters=characters, custom_actions=custom_actions)


In [ ]:
game.game_loop()